In [20]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME - Done #####
#  CRUD Python module file import
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

#### FIX ME - Done #####
# Database username and password 
username = "accuser"
password = "password"

#  instantiate CRUD Python module
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#### FIX ME - Done #####
#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # Grazioso Salvare Logo file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#### FIX ME - Done #####
#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(
    html.Img(
        id='customer-image',
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        alt='customer image',
        style={'height':'10%', 'width':'10%'})
    ),
    #### FIX ME - Done #####
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue or Individual Tracking ', 'value': 'disaster'},
                {'label': 'All', 'value': 'reset'}
            ],
            value='none'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    html.Div(id = 'table-output', children = [
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        style_cell={'textAlign': 'left'},
        style_data={
        'whiteSpace': 'normal',
        'height': 'auto',
        },
        data=df.to_dict('records'),
        #### FIX ME - Done #####
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    ]),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    html.Hr(),
    html.H1("Marisa Kuyava"),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    if (value =='water'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'mountain'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, names = 'breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
#FIXME Add in the code for your geolocation chart
 #### FIX ME - Done #####
    dff = pd.DataFrame.from_dict(viewData)
    location = [dff.location_lat[0], dff.location_long[0]]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=location, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position= location, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app